In [64]:
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import base64
import matplotlib.pyplot as plt
import numpy as np
from elasticsearch import Elasticsearch
from urllib import parse
import pandas as pd
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import logging
logging.basicConfig(level=logging.CRITICAL)

from code_utils.elastic import get_from_es,get_data_from_elastic,plot_graph,interfaces_evaluation
from code_utils.utils import get_year_ipbes,get_year_ipcc,aplatir

In [54]:
body = {
  "size": 0,
  "track_total_hits": True,
  "query": {
    "exists": {
      "field": "topics"
    }
  },
  "aggs": {
    "predicted_labels": {
      "terms": {
        "field": "predict_teds.label.keyword",
        "size": 10000000
      }
    }
  }
}
get_from_es(body,'scanr-publications')

{'took': 306,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 672972, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'predicted_labels': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'ipcc', 'doc_count': 31060},
    {'key': 'ipcc_adaptation', 'doc_count': 20198},
    {'key': 'ipcc_science', 'doc_count': 7183},
    {'key': 'ipcc_mitigation', 'doc_count': 4551}]}}}

In [26]:
body = {
  "size": 0, 
  "query": {
    "exists": {
      "field": "topics"
    }
  }
}

In [84]:
body = {
  "size": 0,
  "track_total_hits": True,
  "query": {
    "exists": {
      "field": "topics"
    }
  },
  "aggs": {
    "years": {
      "terms": {
        "field": "year",
        "size": 10000000
      },
      "aggs": {
        "predicted_labels": {
          "terms": {
            "field": "predict_teds.label.keyword",
            "size": 10000000
          }
        }
      }
    }
  }
}

In [85]:
data=get_from_es(body,'scanr-publications')

In [68]:
years = [x['key'] for x in data]
labels = ['ipcc_adaptation', 'ipcc_science', 'ipcc_mitigation']
values = {label: [] for label in labels}

In [70]:
values

{'ipcc_adaptation': [], 'ipcc_science': [], 'ipcc_mitigation': []}

In [93]:
x=data['aggregations']['years']['buckets'][0]
x

{'key': 2022,
 'doc_count': 117511,
 'predicted_labels': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 'ipcc', 'doc_count': 6508},
   {'key': 'ipcc_adaptation', 'doc_count': 4108},
   {'key': 'ipcc_science', 'doc_count': 1630},
   {'key': 'ipcc_mitigation', 'doc_count': 972}]}}

In [109]:
data_counts={}
for x in data['aggregations']['years']['buckets']:
    print(x['key'])
    data_counts[x['key']]=[x['doc_count']]
    data_counts[x['key']].append([bucket['doc_count'] for bucket in x['predicted_labels']['buckets'] if bucket['key']=='ipcc_science' ][0]) 
    data_counts[x['key']].append([bucket['doc_count'] for bucket in x['predicted_labels']['buckets'] if bucket['key']=='ipcc_adaptation' ][0]) 
    data_counts[x['key']].append([bucket['doc_count'] for bucket in x['predicted_labels']['buckets'] if bucket['key']=='ipcc_mitigation' ][0]) 

2022
2023
2021
2020
2019
2018
2016
2017
2015
2014
2013
2024
2012


IndexError: list index out of range

In [ ]:
for x in data.get('aggregations').get('by_countries').get('buckets'):
    for k in range(len(years)):
        if years[k]== x.get('key'):
            label[years[k]]=x.get('doc_count')
ordered_value={j: round(label[j]*100/tot[j],3) if j in label else 0 for j in groups}
values.append(list(ordered_value.values()))